<h2>Lab 1 - Indice invertido e Busca booleana<h2>

Para esse primeiro lab, vamos usar os conceitos de indice invertido, que vamos será feito com o auxílio de um dicionário, e busca booleana.

<h3>Para esse lab, vamos utilizar as seguintes bibliotecas:<h3>

In [138]:
import pandas as pd
import nltk as nltk

<h2>Dados<h2>

In [183]:
data_antes = pd.read_csv("C:\\Users\\joao_\\Downloads\\noticias_estadao.csv")
#data_depois = pd.read_csv("C:\\Users\\joao_\\Downloads\\noticias_estadao (1).csv") Novo
data_antes.head()

,titulo,conteudo,idNoticia
0,PT espera 30 mil pessoas em festa na Esplanada,BRASÍLIA - Após o desgaste provocado com o lan...,1
1,Alckmin toma posse de olho no Planalto,"Reeleito em outubro, o governador tucano Geral...",2
2,Seis obstáculos e desafios do segundo mandato ...,1. Rearranjo das contas A nova equipe econôm...,3
3,Veja os desafios dos governadores que assumem ...,"No Acre, governador reeleito quer erradicar an...",4
4,PT impulsiona cerimônia de posse da Dilma nas ...,"Os perfis da presidente Dilma Rousseff, nas re...",5


Para receber os asserts do mesmo jeito que foi proposta no canvas, vamos definir uma função para receber e retornar as buscas booleanas para AND e para OR

In [159]:
def search(pesquisa):
    
    pesquisa_split = nltk.word_tokenize(pesquisa.lower())
    
    palavra1 =   pesquisa_split[0]
    operator =   pesquisa_split[1]
    palavra2 =   pesquisa_split[2]
    
    if(operator == 'or'):
        return buscar_or(palavra1,palavra2)
    elif(operator == 'and'):
        return buscar_and(palavra1,palavra2)
    

Para pesquisar pelas palavras presentes no data frame vamos usar dois laços. No primeiro, juntamos o título de cada documento com o conteúdo. Depois disso fazemos um split e, no segundo laço, vamos pesquisar se a palavra que queremos está dentro daquela nova string.

In [152]:
#funcao para buscar as palavras desejadas
id_news = dict()  
def busca_termo(palavra_pesquisada):
    if(palavra_pesquisada in id_news):
        return id_news[palavra_pesquisada]
    else:
        for x in range(data_antes.shape[0]):             #fazer disso uma funcao  usar -> df.shape[0]  
            titulo = data_antes.loc[x,'titulo']         #pego o titulo df.loc -> provavelmente no lab será df.ix
            conteudo = data_antes.loc[x,'conteudo']      #pego o conteudo
            newWorld = titulo +  " " +  conteudo      #concateno os dois separado por espaço
            newNova = nltk.word_tokenize(newWorld.lower())   #faço split usando nltk botando tudo para lower case
            for y in range(len(newNova)):               #faço outro for para percorrer esse novo split procurando a palavra que eu quero
                if(newNova[y] == palavra_pesquisada):  
                    if(palavra_pesquisada in id_news):
                        id_news[palavra_pesquisada].append(data_antes.loc[x,'idNoticia'])              #posso usar o x do primeiro for por que é o mesmo número do ID
                        break;                        #parar para que eu não coloque mais do mesmo indice, caso apareça mais de uma vez
                    else:
                        id_news[palavra_pesquisada] = [data_antes.loc[x,'idNoticia']]
                        break;    
                    
        return id_news[palavra_pesquisada]

Para o algoritmo AND, vamos ultilizar do raciocínio do pseudocódigo visto em sala.

Nesse algoritmo, queremos retornar apenas os documentos em que as duas palavras estão contidas. Para isso, recebemos duas palavras, procuramos por ela e percorremos cada uma dela até alguma acabar, não sendo mais possível ter documentos iguais.

In [169]:
#local onde vamos fazer o AND


def buscar_and(palavra1,palavra2):
    
    iterator1 = 0
    iterator2 = 0
    
    and_answer = []
    
    first_and = busca_termo(palavra1)
    second_and = busca_termo(palavra2)
    
    while(iterator1 < len(first_and) and iterator2 < len(second_and)):
      if(first_and[iterator1] == second_and[iterator2]):
          and_answer.append(first_and[iterator1])
          iterator1 +=1
          iterator2 +=1
      elif(second_and[iterator2] > first_and[iterator1]):
          iterator1 +=1
      else:
          iterator2 += 1
    return and_answer

Para o algorimo OR, vamos ultilizar a mesma idéia de um merge. Iremos fazer isso em 3 laço.

No primeiro laço, percorremos a lista da mesma maneira que percorremos no AND, porém, adicionamos todas as vezes que algum documento for encontrado. Acabado esse laço, não temos a certeza qual das duas listas foi totalmente percorrida. Sendo assim, fazemos mais dois laços que verificam se a lista foi toda percorrida e, se não foi, faz append do resto da sua lista.

In [174]:
#local onde vamos fazer o OR

def buscar_or(palavra1,palavra2):
    
    i = 0
    j = 0
    
    or_answer = []
    
    first = busca_termo(palavra1)
    second = busca_termo(palavra2)
 
    while(i < len(first) and j < len(second)):
        if(first[i] < second[j]):
            or_answer.append(first[i])
            i+= 1
        elif(first[i] > second[j]):
            or_answer.append(second[j])
            j+=1
        else:
            or_answer.append(second[j])
            j+=1
            i+=1
    while(i < len(first)):
        or_answer.append(first[i])
        i+=1
    while(j < len(second)):
        or_answer.append(second[j])
        j+=1
      
    return or_answer



   Para o assert abaixo, tive um pequeno problema. Se eu tivesse usado os dados novos, o tamanho do array  'corruptos' seria 54, enquanto nos dados antigos o tamanho é 53. Por essa diferença, o assert seria 163 e não 164, como descrito no canvas.
   Como já tinha feito todo o código, optei por deixar da mesma maneira.

<h2>Testes<h2>

<h4>AND e OR para debate e presidencial<h4>

In [181]:
assert len(search("debate AND presidencial")) == 201
assert len(search("debate OR presidencial")) == 1770

<h4>AND e OR para presidenciáveis e corruptos<h4>

In [1]:
assert len(search("presidenciáveis AND corruptos")) == 0
assert len(search("presidenciáveis or corruptos")) == 163

NameError: name 'search' is not defined

<h4>AND e OR para Belo e Horizonte<h4>

In [184]:
assert len(search("Belo AND Horizonte")) == 242
assert len(search("Belo OR Horizonte")) == 331